In [1]:
%env XDG_CACHE=/workspace/.cache
%env HF_HOME=/workspace/.cache/huggingface

env: XDG_CACHE=/workspace/.cache
env: HF_HOME=/workspace/.cache/huggingface


In [2]:
from datasets import load_dataset, Dataset
import datasets
import pandas as pd
import random

/workspace/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load dataset
dataset = load_dataset("projecte-aina/teca")
df = dataset['train'].to_pandas()
# Parameters
seed = 55
few_shot_count = 5
random.seed(seed)

entries = {
    "prompt": [],
    "label": [],
    "numeric_label": []
}
labels = {
    1: "neutral",
    0: "entailment",
    2: "contradiction"
}


while len(df) > few_shot_count + 1:
    try:
        examples = []
        examples.append(df.query("label == 0").sample(1, random_state=seed))
        df = df.drop(index=examples[-1].index)
        examples.append(df.query("label == 1").sample(1, random_state=seed))
        df = df.drop(index=examples[-1].index)
        examples.append(df.query("label == 2").sample(1, random_state=seed))
        df = df.drop(index=examples[-1].index)
        examples.append(df.sample(few_shot_count + 1 - 3, random_state=seed))
        df = df.drop(index=examples[-1].index)
    except:
        break
        
    samples = pd.concat(examples)
    
    prompt = ""
    for elm in samples.iloc[:-1].itertuples():
        prompt += f"premise: {elm.premise}\nhypothesis: {elm.hypothesis}\nlabel: {elm.label}"
        prompt += "\n----\n"
        
    last_element = samples.iloc[-1]
    prompt += f"premise: {last_element.premise}\nhypothesis: {last_element.hypothesis}\nlabel: "
    entries["prompt"].append(prompt)
    entries["label"].append(labels[last_element.label])
    entries["numeric_label"].append(last_element.label)


teca = pd.DataFrame(entries)
teca.to_csv("teca.csv", index=False)

In [1]:
import pandas as pd

In [3]:
teca = pd.read_csv("teca.csv")
print(teca.iloc[0]['prompt'])

premise: Nosaltres, per exemple, no estem d'acord de ser la crossa del PSOE.
hypothesis: No volem que el PSOE es recolzi en nosaltres.
label: 0
----
premise: Per una banda es cert que aqui no es reconeix el talent nacional i (entre l'alta burgesia del Liceu i Perelada encara menys).
hypothesis: Tenim veritables talents en l'àmbit musical, especialment els darrers 20 anys.
label: 1
----
premise: Lluís Llach recrimina el paper dels comuns en la votació al parlament del 27 d’octubre
hypothesis: Lluís Llach aplaudeix l'acció dels comuns
label: 2
----
premise: L'ambientació de la novel·la es extraordinària+B951 els paisatges i els personatges són absolutament vius.
hypothesis: Les descripcions de la novel·la són tan vívides que sembla que els paisatges i personatges siguin reals.
label: 0
----
premise: Els llops, bisons, cignes i els falcons també són presents.
hypothesis: Hi ha mamífers i aus.
label: 0
----
premise: La culpa de tot la tenen els altres per vostès, ha criticat.
hypothesis: V

In [4]:
teca

,prompt,label,numeric_label
0,"premise: Nosaltres, per exemple, no estem d'ac...",entailment,0
1,premise: Els CDR convoquen una protesta davant...,contradiction,2
2,premise: Vaig incorporar me a la feina quan es...,contradiction,2
3,premise: La tradició i la modernitat conviuen ...,neutral,1
4,premise: Senda ofereix pràctiques d'un període...,entailment,0
...,...,...,...
2815,"premise: Així, per exemple, ""do-re-mi"" es podr...",contradiction,2
2816,premise: La resposta de l'estat i les administ...,contradiction,2
2817,premise: També volen disfrutar de la neu d'una...,neutral,1
2818,"premise: Sí, a mi també em va atacar un vampir...",contradiction,2
